In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 101 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 25.7 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline
import transformers



In [ ]:
import math
import random
from collections import Counter
from drive.MyDrive.cental_codes.TSAR2022SharedTaskmain import tsar_eval

In [ ]:
all_ds = {}
for ds in ["/content/drive/MyDrive/cental_codes/TSAR2022SharedTaskmain/datasets/trial/tsar2022_en_trial_gold.tsv",
           "/content/drive/MyDrive/cental_codes/TSAR2022SharedTaskmain/datasets/trial/tsar2022_es_trial_gold.tsv",
           "/content/drive/MyDrive/cental_codes/TSAR2022SharedTaskmain/datasets/trial/tsar2022_pt_trial_gold.tsv",
           ]:
    with open(ds) as inputfile:
        lang = ds .split("/")[-1].split("_")[1]
        all_ds[lang] = []
        for ln in inputfile:
            cols = ln.strip().split("\t")
            sent = cols[0]
            target = cols[1]
            reply = Counter(cols[2:])
            assert target in sent
            assert len(sent.split(target)) == 2
            all_ds[lang].append({"sent": sent, "target": target, "ans": None, "gold": None, "ds_name": ds})



# ALEXIS-LSBert: bert + roberta

In [ ]:
bert_models = {"es":[
                        "roberta-base", 
                        "roberta-large", 
                        "skimai/spanberta-base-cased", 
                        "PlanTL-GOB-ES/roberta-base-bne",
                        "dccuchile/bert-base-spanish-wwm-cased",    #bert
                        "dccuchile/bert-base-spanish-wwm-uncased"    #bert
                     
                    ],
               "pt":[
                        "josu/roberta-pt-br",
                        "rdenadai/BR_BERTo",
                        "neuralmind/bert-base-portuguese-cased",    #bert
                        "neuralmind/bert-large-portuguese-cased",    #bert
                    ],
               "en":[
                        "roberta-base",
                        "roberta-large",
                        "bert-large-uncased",    #bert
                        "bert-base-uncased"    #bert
                    ],
                }

bert_models

{'en': ['roberta-base',
  'roberta-large',
  'bert-large-uncased',
  'bert-base-uncased'],
 'es': ['roberta-base',
  'roberta-large',
  'skimai/spanberta-base-cased',
  'PlanTL-GOB-ES/roberta-base-bne',
  'dccuchile/bert-base-spanish-wwm-cased',
  'dccuchile/bert-base-spanish-wwm-uncased'],
 'pt': ['josu/roberta-pt-br',
  'rdenadai/BR_BERTo',
  'neuralmind/bert-base-portuguese-cased',
  'neuralmind/bert-large-portuguese-cased']}

In [ ]:
all_predictions = {}
for lang in all_ds:
    print('Lang',lang)
    predictions = {m:[] for m in bert_models[lang]}
    ds_file = None
    results = {}
    for model_name in bert_models[lang]:
        print(lang,model_name)
        bert = pipeline('fill-mask',model=model_name, use_fast=True, top_k=50)
        for case in all_ds[lang]:
            # sequence_to_classify = case["sent"].strip() + " [SEP] "+ case["sent"].replace(case["target"], "[MASK]" if "bert-" in model_name else "<mask>")
            sequence_to_classify = case["sent"].strip() + " "+ case["sent"].replace(case["target"], "[MASK]" if "bert-" in model_name else "<mask>")
            ans = bert(sequence_to_classify)
            # lst = [x.strip().lower() for _,x in sorted([(v["score"],v["token_str"]) for v in ans], reverse = True)]
            
            predictions[model_name].append({"sent":case["sent"],"target":case["target"], "gold":case["gold"], "ans":sorted([(v["score"],v["token_str"]) for v in ans], reverse = True)})
            ds_file = case["ds_name"]
    all_predictions[lang] = {"ds_file":ds_file, "predictions":predictions}
    # result = evaluate(ds_file, predictions)


Lang en
en roberta-base


en roberta-large


en bert-large-uncased


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


en bert-base-uncased


...

pt rdenadai/BR_BERTo


pt neuralmind/bert-base-portuguese-cased


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt neuralmind/bert-large-portuguese-cased


Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
all_predictions2 = {}
for lang in all_predictions:
    predictions = all_predictions[lang]["predictions"]
    ds_file = all_predictions[lang]["ds_file"]
    predictions_aux = {"ds_file":ds_file, "predictions":{}}
    for model in predictions:
        print(lang,model)
        for case in predictions[model]:
            k = case["sent"] + "_-_" +  case["target"]
            if k not in predictions_aux["predictions"]:
                predictions_aux["predictions"][k] = {m:None for m in predictions}
            if predictions_aux["predictions"][k][model] != None:
                print(predictions_aux["predictions"][k][model])
            assert predictions_aux["predictions"][k][model] == None
            predictions_aux["predictions"][k][model] = case
    all_predictions2[lang] = predictions_aux

    

en roberta-base
en roberta-large
en bert-large-uncased
en bert-base-uncased
es roberta-base
es roberta-large
es skimai/spanberta-base-cased
es PlanTL-GOB-ES/roberta-base-bne
es dccuchile/bert-base-spanish-wwm-cased
es dccuchile/bert-base-spanish-wwm-uncased
pt josu/roberta-pt-br
pt rdenadai/BR_BERTo
pt neuralmind/bert-base-portuguese-cased
pt neuralmind/bert-large-portuguese-cased


In [ ]:
sep = "\t"
for lang in all_predictions2:
    with open("resp_"+lang+"_v2b.csv","w") as output_file:
        predictions = all_predictions2[lang]["predictions"]
        # ds_file = all_predictions2[lang]["ds_file"]
        # predictions_to_eval = []
        for index, sent in enumerate(predictions):
            ans = []
            d = {}
            for model in predictions[sent]:
                case = predictions[sent][model]
                # print(case["ans"])
                ans.extend(case["ans"])
                for w in case["ans"]:
                    if w[1] not in d:
                        d[w[1]] = 0
                    d[w[1]] += w[0]
            cnt = Counter([v for s,v in ans]).most_common(len([v for s,v in ans]))
            # ans = [(d[w]/f, w) for w,f in cnt]
            ans = [(d[w], w) for w,f in cnt]
            ans = [w for s,w in sorted(ans, reverse=True)]
            # print(index, [1 if w in case["gold"] else 0 for w, f in cnt], "\t--\t", [w for w, f in cnt])
            # output_file.write(case["sent"] +sep+case["target"]+sep+ sep.join(ans[:12]) + "\n")
            output_file.write(case["sent"] +sep+case["target"]+sep+ sep.join(ans) + "\n")



# NLI

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
for lang in ["en", "es", "pt"]:
    input_file_name = "resp_"+lang+"_v1.csv"
    print('Lang',lang)
    output_file = open(input_file_name.replace(".csv", "_50v2_nli.csv"), "w")
    for i, ln in enumerate(open(input_file_name).readlines()):
        cols = ln.strip().split("\t")
        sent = cols[0]
        target = cols[1]
        cand = cols[2:50]

        ans = classifier(sent, cand)
        lst = [x for _,x in sorted(zip(ans["scores"],ans["labels"]), reverse = True)]
        print('\t',i, lst)
        output_file.write(sent+sep+target+sep+sep.join(lst) + "\n")
    output_file.close()

Lang en
	 0 ['available', 'optional', ' compulsory', 'possible', 'prescribed', ' mandated', 'compulsory', 'mandated', ' obligatory', ' optional', ' required', ' mandatory', 'required', 'obligatory', '##rricular', 'difficult', 'mandatory', ' obliged', 'permitted', ' forced', 'voluntary', ' voluntary', 'appropriate', ' necessary', 'necessary', 'proper', 'affordable', 'expensive', 'forbidden', 'standard', 'easier', 'essential', 'exclusively', 'normal', 'enough', ' uniform', 'simple', ' sufficient', 'only', 'sufficient', 'customary', 'easy', 'commonplace', ' routine', 'ordinary', 'unnecessary', 'illegal', 'impossible']
	 1 ['injected', 'seeded', ' injected', 'infused', 'created', 'introduced', 'demonstrated', ' infused', 'presented', 'identified', 'addressed', 'affected', 'set', 'empowered', 'struck', 'established', 'provided', 'supplied', 'delivered', 'governed', 'bred', 'endowed', 'treated', 'prepared', 'produced', ' endowed', 'furnished', 'improved', 'developed', 'trained', ' inst', ' f

# parser filtered

In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 4.9 MB/s 
     |████████████████████████████████| 197 kB 48.8 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=1ec32824b02a4925fd469340c2d0437e4301b201bc2af0f33a970b9e445e416d
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
import stanza

In [ ]:
stanza.download("en")
stanza.download("es")
stanza.download("pt")

INFO:stanza:Downloading default packages for language: en (English)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:Downloading default packages for language: es (Spanish)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:Downloading default packages for language: pt (Portuguese)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
for lang in ["pt", "en", "es"]:
    input_file_name = "resp_"+lang+"_v1.csv"
    print('Lang',lang)
    nlp = stanza.Pipeline(lang=lang, processors='tokenize,mwt,pos', tokenize_pretokenized=True)
    output_file = open(input_file_name.replace(".csv", "_10Pr_nli.csv"), "w")
    for i, ln in enumerate(open(input_file_name).readlines()):
        cols = ln.strip().split("\t")
        sentO = cols[0]
        target = cols[1]
        p1, p2 = sentO.split(target)
        id = None
        pos = None
        # print()
        aux = [w for w in p1.split(" ") + [target] + p2.split(" ") if len(w)>0]
        # print(aux)
        doc = nlp([aux])
        for sent in doc.sentences:
            for word in sent.words:
                # print(word.text,word.text == target)
                if word.text == target:
                    id = word.id
                    pos = word.upos
        cand = set([w.strip() for w in cols[2:] if not w.startswith("##")])
        assert id 
        assert pos
        good_cand = []
        for c in cand:
            if len(c)==0:
                continue
            aux = [w for w in p1.split(" ") + [c] + p2.split(" ") if len(w)>0]
            doc = nlp([aux])
            w = [word.upos for sent in doc.sentences for word in sent.words if word.id == id]
            if len(w)==0:
                print(aux)
                print(id,c)
                print(doc)
            w = w[0]
            if w == pos:
                good_cand.append(c)
        assert len(good_cand)>1
        ans = classifier(sentO, good_cand[:10])
        lst = [x for _,x in sorted(zip(ans["scores"],ans["labels"]), reverse = True)]
        print('\t',i, len(lst), lst)
        output_file.write(sentO+sep+target+sep+sep.join(lst) + "\n")
    output_file.close()


# for lang in ["en", "es", "pt"]:
#     input_file_name = "resp_"+lang+"_v1.csv"
#     print('Lang',lang)
#     output_file = open(input_file_name.replace(".csv", "_50_nli.csv"), "w")
#     for i, ln in enumerate(open(input_file_name).readlines()):
#         cols = ln.strip().split("\t")
#         sent = cols[0]
#         target = cols[1]
#         cand = set([w.strip() for w in cols[2:]])
#         p1, p2 = sent.split(target)


#         ans = classifier(sent, cand)
#         lst = [x for _,x in sorted(zip(ans["scores"],ans["labels"]), reverse = True)]
#         print('\t',i, lst)
#         output_file.write(sent+sep+target+sep+sep.join(lst) + "\n")
#     output_file.close()

Lang pt


INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


	 0 10 ['fazendo', 'passando', 'observando', 'deixando', 'formando', 'causando', 'tomando', 'liberando', 'segurando', 'permanecendo']
	 1 10 ['projetos', 'eletrônicos', 'fluidos', 'ativos', 'planos', 'remédios', 'fatores', 'programas', 'fios', 'medicamentos']
	 2 10 ['recuperar', 'atender', 'criar', 'acompanhar', 'convencer', 'revelar', 'apurar', 'identificar', 'pegar', 'vencer']
	 3 10 ['agressão', 'ave', 'tragédia', 'qualidade', 'criatura', 'delas', 'borboleta', 'região', 'águia', 'epidemia']
	 4 10 ['fazendo', 'passando', 'contribuem', 'enviados', 'trabalhando', 'enviando', 'deixando', 'saindo', 'indo', 'lutando']
	 5 10 ['discutido', 'visto', 'utilizado', 'entregue', 'proposto', 'anunciado', 'mostrado', 'votado', 'aceite', 'removido']
	 6 10 ['representação', 'expressão', 'diferenciação', 'cooperação', 'exploração', 'aceitação', 'totalidade', 'consciência', 'hierarquia', 'mistura']
	 7 10 ['afirmam', 'levam', 'contribuem', 'tendem', 'determina', 'criam', 'garante', 'atingem', 'afet

INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


	 0 10 ['requisite', 'bureaucratic', 'legal', 'lifelong', 'redundant', 'punitive', 'bourgeois', 'continental', 'simple', 'unnecessary']
	 1 10 ['introduced', 'aroused', 'struck', 'established', 'refreshed', 'challenged', 'hit', 'chilled', 'impoverished', 'illed']
	 2 10 ['puppets', 'enemies', 'crimes', 'fighters', 'terror', 'comrades', 'dealers', 'remnants', 'hunters', 'mercenaries']
	 3 10 ['witnesses', 'observation', 'spectators', 'bystanders', 'organizers', 'inspections', 'reinforcements', 'fighters', 'foreigners', 'tourists']
	 4 10 ['caliber', 'shock', 'debris', 'battle', 'shell', 'cartridge', 'shoulder', 'roadside', 'grenade', 'linebacker']
	 5 10 ['shrouded', 'presented', 'emerged', 'described', 'concealed', 'disgu', 'speaking', 'wearing', 'trained', 'imprisoned']
	 6 10 ['issue', 'alternative', 'entity', 'indication', 'initiative', 'adherence', 'excess', 'shoot', 'overhaul', 'outpost']
	 7 10 ['Musical', 'philharmonic', 'formal', 'general', 'harmonic', 'continental', 'canonical

INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


	 0 10 ['fundamental', 'característico', 'sujetas', 'formal', 'contemporáneo', 'pintoresco', 'euskera', 'general', 'infantil', 'uruguayo']
	 1 10 ['conocido', 'difunto', 'probable', 'dominante', 'super', 'ástico', 'viejo', 'póstumo', 'unico', 'peor']
	 2 10 ['suscribirse', 'define', 'prepararse', 'transmit', 'cumplir', 'oponerse', 'atender', 'cular', 'icó', 'centaje']
	 3 10 ['unfavorable', 'perdidos', 'inadecuados', 'identificados', 'probable', 'fanáticos', 'fice', 'fals', 'perfectos', 'durable']
	 4 10 ['elevada', 'cualificada', 'legítima', 'fuerte', 'determinada', 'llamarnos', 'ollo', 'sólida', 'grave', 'estupenda']
	 5 10 ['superaron', 'ocuparon', 'obtuvo', 'establecieron', 'cumplir', 'desarrollaron', 'hit', 'comprob', 'uso', 'battery']
	 6 10 ['vista', 'acepte', 'descubra', 'constituya', 'resume', 'fije', 'desarrolle', 'TM', 'google', 'olvide']
	 7 10 ['presente', 'mencionada', 'contemporánea', 'difunto', 'investi', 'super', 'originaria', 'pseudo', 'restante', 'destruida']
	 8 10 

In [ ]:
trim + set + filtrar POS